In [1]:
%%capture
!pip install trl peft bitsandbytes accelerate evaluate tensorboard datasets

In [2]:
exit()

# I. prepare dataset

In [2]:
from datasets import load_dataset
# Convert dataset to OAI messages
system_message = """Bạn là một trợ lý hữu ích, tôn trọng và trung thực. Luôn trả lời một cách hữu ích nhất có thể trong khi vẫn an toàn. Câu trả lời của bạn không được bao gồm bất kỳ nội dung có hại, phi đạo đức, phân biệt chủng tộc, phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp. Hãy đảm bảo rằng câu trả lời của bạn mang tính chất tích cực và không thiên vị về mặt xã hội.\n\nNếu một câu hỏi không có ý nghĩa gì hoặc không mạch lạc về mặt thực tế, hãy giải thích lý do thay vì trả lời điều gì đó không chính xác. Nếu bạn không biết câu trả lời cho một câu hỏi, vui lòng không chia sẻ thông tin sai lệch.
{schema}"""

def create_conversation(sample):
  return {
    "messages": [
      {"role": "system", "content": system_message.format(schema=sample["instruction"])},
      {"role": "user", "content": sample["input"]},
      {"role": "assistant", "content": sample["output"]}
    ]
  }

# Load dataset from the hub
dataset = load_dataset("nlplabtdtu/qa_vi_small", split="train")

# Convert dataset to OAI messages
dataset = dataset.map(create_conversation, remove_columns=dataset.features,batched=False)
# split dataset into 10,000 training samples and 2,500 test samples
dataset = dataset.train_test_split(test_size=2500/12500)

print(dataset["train"][345]["messages"])

# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

[{'content': 'Bạn là một trợ lý hữu ích, tôn trọng và trung thực. Luôn trả lời một cách hữu ích nhất có thể trong khi vẫn an toàn. Câu trả lời của bạn không được bao gồm bất kỳ nội dung có hại, phi đạo đức, phân biệt chủng tộc, phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp. Hãy đảm bảo rằng câu trả lời của bạn mang tính chất tích cực và không thiên vị về mặt xã hội.\n\nNếu một câu hỏi không có ý nghĩa gì hoặc không mạch lạc về mặt thực tế, hãy giải thích lý do thay vì trả lời điều gì đó không chính xác. Nếu bạn không biết câu trả lời cho một câu hỏi, vui lòng không chia sẻ thông tin sai lệch.\nXác định phương pháp nên được sử dụng để giải phương trình này:', 'role': 'system'}, {'content': '2y + 6 = 10', 'role': 'user'}, {'content': 'Phép trừ và phép chia. Trước tiên, chúng ta trừ 6 từ cả hai phía, sau đó, chúng ta chia cho 2. Kết quả là y = 2.', 'role': 'assistant'}]


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

383456

In [3]:
from datasets import load_dataset

# Load jsonl data from disk
dataset = load_dataset("json", data_files="train_dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

## Setup model

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Hugging Face model id
model_id = "sail/Sailor-1.8B-Chat"
tokenizer_id = "sail/Sailor-1.8B-Chat"


# BitsAndBytesConfig int-4 config
# Ref: https://huggingface.co/docs/transformers/main/main_classes/quantization#transformers.BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, #enable 4-bit quantization
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
tokenizer.padding_side = 'right' # to prevent warnings


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear4bit(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMS

In [6]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
# Ref: https://huggingface.co/docs/peft/main/en/package_reference/lora#peft.LoraConfig
peft_config = LoraConfig(
        lora_alpha=8,
        lora_dropout=0.05,
        r=6,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)


In [7]:
from transformers import TrainingArguments


# doc: https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments

args = TrainingArguments(
    output_dir="chat", # directory to save and repository id
    num_train_epochs=1,                     # number of training epochs
    per_device_train_batch_size=2,          # batch size per device during training
    gradient_accumulation_steps=2,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
#     bf16=True,                              # use bfloat16 precision
#     tf32=True,                              # use tf32 precision
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=False,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
)


In [8]:
from trl import SFTTrainer

max_seq_length = 1024 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    dataset_kwargs={
        "add_special_tokens": False, # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
    }
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, packing, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:192: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:366: UserWarning: You passed a `dataset_kwargs` argument to the SFTTrainer, the value you passed will override the o

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
trainer.train()
trainer.save_model()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.110400
20,0.817800
30,0.656800
40,0.692000
50,0.644300


In [10]:
from huggingface_hub import login
login()

In [11]:
# # push model and tokenizer to hub
# model.push_to_hub('nlplabtdtu/Qwen-qa')
# tokenizer.push_to_hub('nlplabtdtu/Qwen-qa')

model.safetensors:   0%|          | 0.00/1.64G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nlplabtdtu/Qwen-qa/commit/01ec100057dfd06378af7993a04b4e3f150574a4', commit_message='Upload tokenizer', commit_description='', oid='01ec100057dfd06378af7993a04b4e3f150574a4', pr_url=None, pr_revision=None, pr_num=None)

## Dự đoán mô hình

In [10]:
from transformers import pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
# get token id for end of conversation
eos_token = tokenizer("<|im_end|>",add_special_tokens=False)["input_ids"][0]

In [11]:
tokenizer.eos_token

'<|endoftext|>'

In [14]:
prompts = [
    "Viết cho tôi thuật toán tìm số lớn nhất trong một mảng cho trước.",
    "Làm sao để tôi có thể có một giấc ngủ ngon.",
    "Dựa vào tiêu đề sau hãy viết cho tôi một đoạn văn mô tả sản phẩm này: Dầu gôi bạc hà clear.",
    "1+1="
]

def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    outputs = pipe(
        prompt,
        max_new_tokens=128,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        eos_token_id=eos_token,
        pad_token_id=eos_token,
    )
    return outputs[0]['generated_text'][len(prompt):].strip()


for prompt in prompts:
    print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(prompt)}")
    print("-"*50)



    prompt:
Viết cho tôi thuật toán tìm số lớn nhất trong một mảng cho trước.
    response:
Thực ra, để tìm số lớn nhất trong một mảng cho trước, bạn có thể sử dụng một mảng có chứa các số lớn nhất đã được biết trước đó. Sau đó, bạn có thể tìm kiếm các số đó trong mảng và lấy số lớn nhất. Nếu bạn không có dữ liệu về các số lớn nhất, bạn có thể sử dụng một truy vấn tìm kiếm để tìm số lớn nhất cho trước đó.
--------------------------------------------------
    prompt:
Làm sao để tôi có thể có một giấc ngủ ngon.
    response:
Nếu bạn gặp khó khăn trong việc ngủ, hãy thử làm theo những lời khuyên sau đây:

1. Tạo một thói quen ngủ lành mạnh: Dành 30 phút cho việc luyện tập trước khi đi ngủ, hoặc dành 30 phút cho các hoạt động thư giãn trước khi đi ngủ. Luôn đảm bảo rằng bạn đi ngủ đủ 7-8 giờ mỗi ngày.

2. Tránh caffeine: Nếu bạn uống cà phê, trà, hoặc các đồ uống có caffein khác vào buổi tối, hãy tránh chúng ít nhất một giờ trước khi đi ngủ.

3. Giảm căng thẳng:
--------------------------